Импортирование нужных библиотек

In [1]:
# !pip install pyLDAvis
# !pip install pymorphy2-dicts
# !pip install stop-words
# !pip install pymorphy3
#

In [25]:
import gensim
import pandas as pd
import numpy as np
import nltk
import re
import pprint
import pymorphy2
import pymorphy3
import warnings
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora
from stop_words import get_stop_words
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oleg1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Считывание корпуса в массив и вычисление длины корпуса

In [26]:
import os

folder_path = "../corpus/"
corpus = []
document_names = []

def read_documents_from_folder(folder_path, documents_per_folder=None):
    documents_read = 0
    
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".txt") and (documents_per_folder is None or documents_read < documents_per_folder):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    text = text.replace("&raquo;", " ").replace("&laquo;", " ").replace("&mdash;", " ").replace("&nbsp;", " ")
                    corpus.append(text)
                    document_names.append(filename)
                    documents_read += 1

documents_per_folder = 200


#for subfolder in ["Music", "Fashion", "Food"]:
for subfolder in ["Sport", "Cybersport"]:
    subfolder_path = os.path.join(folder_path, subfolder)
    read_documents_from_folder(subfolder_path, documents_per_folder)


In [27]:
len(corpus)

400

Расчет минимальной, средней и минимальной длины документа в корпусе

In [28]:
if len(corpus) > 0:
    total_words = 0
    min_words = float('inf')
    max_words = 0
    min_document_name = ""
    max_document_name = ""

    for document, document_name in zip(corpus, document_names):
        words = word_tokenize(document)
        num_words = len(words)
        total_words += num_words

        if num_words < min_words:
            min_words = num_words
            min_document_name = document_name
        if num_words > max_words:
            max_words = num_words
            max_document_name = document_name

    average_words = total_words / len(corpus)

    print(f"Среднее количество слов в документе: {average_words:.2f}")
    print(f"Минимальное количество слов в документе: {min_words} (документ: {min_document_name})")
    print(f"Максимальное количество слов в документе: {max_words} (документ: {max_document_name})")
else:
    print("Корпус документов пуст.")

Среднее количество слов в документе: 1181.62
Минимальное количество слов в документе: 489 (документ: 189.txt)
Максимальное количество слов в документе: 3150 (документ: 128.txt)


Токенизация, нормализация, фильтрация текста

In [29]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []

    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        if token == 'dota' and i + 1 < len(tokens) and tokens[i + 1] == '2':
            preprocessed_text.append('dota 2')
            i += 2
        elif token == 'cs' and i + 1 < len(tokens) and tokens[i + 1] == 'go':
            preprocessed_text.append('CS:GO')
            i += 2
        elif token == 'league' and i + 1 < len(tokens) and tokens[i + 1] == 'of' and i + 2 < len(tokens) and tokens[i + 2] == 'legends':
            preprocessed_text.append('League of Legends')
            i += 3
        elif ":" in token and all(part.isdigit() for part in token.split(":")):
            i += 1
        else:
            if token not in punctuation_marks:
                if not token.isdigit():
                    lemma = morph.parse(token)[0].normal_form
                    if lemma not in stop_words:
                        preprocessed_text.append(lemma)
            i += 1

    return preprocessed_text

In [30]:
stop_words = get_stop_words('ru')
punctuation_marks = ['!', ',', '(', ')',"&", '&', "the","cdec", "eg", "Reuters", "wesg", "фото",'Фото', "дот", "\'\'", ':', "``", "`", "например", '-', '?', '.',  '[', ']', '..', '...', '«', '»', ';', '–', '--', "—", '$', "ст", "л", '%', "год", "лука", "лук", "тест", "это", "один", "самый", "который", "ст.ть"]
morph = pymorphy3.MorphAnalyzer()

In [31]:
preprocessed_corpus = []

for text in corpus:
    preprocessed_text = preprocess(text, punctuation_marks, stop_words, morph)
    preprocessed_corpus.append(preprocessed_text)

Подсчет количества каждого слова в во всем корпусе и удаление слов, встречающихся больше 2 раз

In [32]:
from collections import defaultdict

frequency = defaultdict(int)
for text in preprocessed_corpus:
    for token in text:
        frequency[token] += 1

In [33]:
processed_corpus1 = [[token for token in text if frequency[token] > 2] for text in preprocessed_corpus]

Составление словаря корпуса и векторизация с помощью Doc2Bow

In [34]:
dictionary = corpora.Dictionary(processed_corpus1)

In [35]:
corpus = [dictionary.doc2bow(text) for text in processed_corpus1]

Применение модели LDA

In [36]:
from gensim.models import LdaModel

num_topics = 2
num_iterations = 30

ldamodel = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=num_iterations)

In [37]:
ldamodel.show_topics()

[(0,
  '0.015*"матч" + 0.009*"команда" + 0.009*"лига" + 0.007*"чемпион" + 0.006*"первый" + 0.006*"игра" + 0.006*"сезон" + 0.005*"мяч" + 0.005*"гол" + 0.004*"клуб"'),
 (1,
  '0.022*"команда" + 0.013*"team" + 0.011*"турнир" + 0.010*"игра" + 0.009*"игрок" + 0.006*"место" + 0.006*"первый" + 0.006*"играть" + 0.006*"international" + 0.006*"хороший"')]

In [38]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [39]:
#document = "Овощи – это не только вкусно, но и невероятно полезно для нашего здоровья. Богатые витаминами, минералами и антиоксидантами, они являются основой здорового питания и способствуют поддержанию оптимального функционирования организма. 1. Богатство Витаминами и Минералами:Овощи предоставляют нам широкий спектр витаминов и минералов, необходимых для поддержания жизнедеятельности. Витамин С, содержащийся в брокколи и красном перце, способствует иммунной системе и укрепляет костную ткань. Фолиевая кислота в шпинате необходима для нормального развития клеток и поддержания здоровья сердца. 2. Антиоксиданты для Борьбы с Свободными Радикалами:Овощи являются отличным источником антиоксидантов, которые защищают наши клетки от воздействия свободных радикалов. Такие овощи, как помидоры и морковь, содержат бета-каротин, который способствует замедлению процессов старения и поддерживает здоровье кожи. 3. Волокно для Здоровья Пищеварительной Системы: Пищевое волокно, содержащееся в овощах, играет ключевую роль в поддержании здоровья пищеварительной системы. Благодаря волокну, овощи помогают предотвращать запоры и поддерживают нормальную работу кишечника. 4. Снижение Риска Заболеваний: Регулярное употребление овощей связано с снижением риска различных заболеваний, включая сердечно-сосудистые и онкологические заболевания. Овощи богаты антиоксидантами, которые могут помочь в защите организма от различных заболеваний.5. Стимуляция Потребления Воды:Многие овощи, такие как огурцы и томаты, содержат высокий процент воды, способствуя увлажнению организма. Это особенно важно для поддержания правильного баланса воды и общего здоровья. Овощи – это неотъемлемая часть сбалансированного рациона. Интегрирование разнообразных овощей в ежедневное питание поможет вам на пути к здоровому образу жизни и полноценному питанию. Не забывайте о разнообразии и цветовой палитре овощей, чтобы получить максимальную пользу для вашего организма."
document = "Киберспорт в Dota 2 представляет собой соревновательную деятельность, связанную с популярной многопользовательской онлайн-игрой Dota 2, разработанной компанией Valve Corporation. Dota 2 — это стратегическая игра в жанре MOBA (многопользовательская онлайн-боевая арена), где две команды, состоящие из пяти игроков каждая, сражаются за контроль над игровым полем, строениями и ресурсами. Киберспортивные события в Dota 2 привлекают внимание миллионов зрителей по всему миру. Турниры проводятся на различных уровнях, начиная от местных чемпионатов до международных турниров высшего уровня, таких как The International. Команды, состоящие из выдающихся игроков, соперничают за призовые фонды, которые могут достигать многомиллионных долларов. Каждая команда в Dota 2 имеет свой уникальный стиль игры, стратегии и состав игроков. Киберспортивные матчи представляют собой настоящие тактические сражения, где решающую роль играют не только мастерство отдельных игроков, но и их способность работать в команде. В связи с ростом популярности киберспорта в Dota 2 формируются профессиональные игровые организации, тренерские штабы и фан-сообщества, что подчеркивает влияние киберспорта на современную культуру и развлечения. Подобное описание можно адаптировать и для других популярных киберспортивных игр, таких как Counter-Strike: Global Offensive (CS:GO), League of Legends и других."

document_bow = dictionary.doc2bow(document.split())
document_topics = ldamodel.get_document_topics(document_bow)

print(document_topics)

[(0, 0.03481791), (1, 0.96518207)]


In [40]:
warnings.filterwarnings("ignore")

pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.120984  0.0       1        1  61.244653
0     -0.120984  0.0       2        1  38.755347, topic_info=               Term         Freq        Total Category  logprob  loglift
6549           team  1760.000000  1760.000000  Default  30.0000  30.0000
91             матч  2088.000000  2088.000000  Default  29.0000  29.0000
85             лига   886.000000   886.000000  Default  28.0000  28.0000
102             мяч   451.000000   451.000000  Default  27.0000  27.0000
6729  international   843.000000   843.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
145       последний   271.108528   746.230283   Topic2  -5.7943  -0.0646
458         хороший   283.006221  1097.358810   Topic2  -5.7514  -0.4073
68           играть   247.870650  1098.002013   Topic2  -5.8839  -0.5404
231       чемпионат   230.642143   475.211574   Topic2  -5.9560   0.2250
92            место   240.886221  1110.747218   Topic2  -5.9125  -0.5806

[143 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
476       1  0.012538        1-й
476       2  0.986342        1-й
6800      1  0.999349      2023.
6569      1  0.999532    betboom
4389      1  0.996792         cs
...     ...       ...        ...
462       1  0.279182    чемпион
462       2  0.720623    чемпион
231       1  0.515560  чемпионат
231       2  0.486099  чемпионат
951       2  0.998020      шайба

[158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

Применение модели LSI

In [41]:
from gensim.models import LsiModel

warnings.filterwarnings("ignore", category=DeprecationWarning, module="scipy.linalg._special_matrices")
warnings.filterwarnings("ignore", message="`scipy.sparse.sparsetools.csc_matvecs` is deprecated.*")


num_topics = 2

lsimodel = LsiModel(corpus, id2word=dictionary, num_topics=num_topics)

In [42]:
lsimodel.show_topics(num_topics=2, num_words=12, log=False, formatted=True)

[(0,
  '0.497*"команда" + 0.314*"team" + 0.207*"турнир" + 0.197*"игра" + 0.174*"игрок" + 0.158*"матч" + 0.149*"играть" + 0.142*"место" + 0.139*"первый" + 0.132*"international" + 0.131*"хороший" + 0.126*"смочь"'),
 (1,
  '-0.412*"team" + 0.365*"матч" + 0.242*"лига" + 0.159*"чемпион" + 0.139*"мяч" + -0.134*"betboom" + -0.131*"spirit" + -0.127*"international" + 0.126*"игрок" + 0.122*"первый" + -0.122*"esports" + 0.121*"тренер"')]

Применение модели NNMF

In [43]:
from gensim.models import Nmf

warnings.filterwarnings("ignore", category=DeprecationWarning, module="gensim")

num_topics = 2
passes = 300

nmf = Nmf(corpus, num_topics=num_topics, passes=passes)

for topic_id in range(num_topics):
    topic_words = nmf.show_topic(topic_id)
    words = [dictionary[int(word)] for word, _ in topic_words]
    print(f"Тема {topic_id + 1}: {', '.join(words)}")


Тема 1: матч, команда, лига, игра, игрок, первый, чемпион, сезон, тренер, мяч
Тема 2: команда, team, турнир, игра, international, играть, место, esports, игрок, смочь
